In [0]:
USE redox.himss25;

In [0]:
SELECT parse_json(ai_query(
  'redox_make_request',
  request => named_struct(
    'http_method', 'post'
    ,'resource', 'Observation'
    ,'action', '\$observation-create'
    ,'data', '
{
   "resourceType":"Bundle",
   "entry":[
      {
         "resource":{
            "category":[
               {
                  "coding":[
                     {
                        "code":"survey",
                        "display":"Survey",
                        "system":"http://terminology.hl7.org/CodeSystem/observation-category"
                     }
                  ]
               }
            ],
            "code":{
               "coding":[
                  {
                     "code":"78033-8",
                     "display":"Remaining Hospital Stay",
                     "system":"http://loinc.org"
                  }
               ],
               "text":"Remaining Hospital Stay"
            },
            "effectiveDateTime":"2024-01-28T18:06:33.245-05:00",
            "issued":"2024-01-28T18:06:33.245-05:00",
            "resourceType":"Observation",
            "status":"final",
            "valueQuantity":{
               "code":"days",
               "system":"https://www.nubc.org/CodeSystem/RevenueCodes",
               "unit":"days",
               "value":4
            },
            "subject": {
              "reference": "Patient/58117110-ae47-452a-be2c-2d82b3a9e24b"
            },
            "identifier": [
            {
              "system": "urn:databricks",
              "value": "1234567890"
            }
          ]
         }
      },
      {
         "resource":{
           "resourceType": "Patient",
           "identifier": [
            {
              "system": "urn:redox:health-one:MR",
              "value": "0000991458"
            },
            {
              "system": "http://hl7.org/fhir/sid/us-ssn",
              "value": "547-01-9991"
            }
          ]
         }
      }
   ]
}
')
,returnType => "STRING")) as result

In [0]:
CREATE OR REPLACE FUNCTION redox_request 
(
  http_method STRING COMMENT "The http method to invoke for the Redox Rest API. Typically 'post' or 'get'." 
  ,resource STRING COMMENT "The Redox resource endpoint to use for the REST API. (Note including the base_url.)" 
  ,action STRING COMMENT "The Redox action to invoke for the resource. (e.g. For ClaimResponse, the action is <dollar_sign>respond.)" 
  ,data STRING COMMENT "The data to send to the Redox API. This is typically a JSON string but may be a base64 encoded string." 
)
RETURNS VARIANT
LANGUAGE SQL
NOT DETERMINISTIC
CONTAINS SQL
COMMENT 'The response text from the Redox API Call.'
RETURN try_parse_json(
  ai_query(
    endpoint => 'redox_make_request'
    ,request => named_struct(
      'http_method', http_method
      ,'resource', resource
      ,'action', action
      ,'data', data
    )
    ,returnType => "STRING"
  )
);

In [0]:
with eg as (
  SELECT
    "post" as http_method
    ,"DiagnosticReport" as resource
    ,"_search" as action
    ,"subject=Patient/81c2f5eb-f99f-40c4-b504-59483e6148d7" as data
)
SELECT
  redox_request(
    http_method
    ,resource
    ,action
    ,data
  )
FROM 
  eg
;